In [1]:
from random import *
from math import *
from time import time

In [27]:
# Getting input from a file
start_states = []
goal = []
heuristics = {}

with open("input_file.txt") as f:
        for (i,line) in enumerate(f):
            if(i==0):
                list1 = line.split(':')[1:]
                start_states = [list(map(int, state.split(','))) for state in list1]
            if(i==1):
                list1 = line.split(':')[1:]
                goal = [list(map(int, state.split(','))) for state in list1]
            if(i==2):
                list1 = (line.split(':'))[1].split(',')
                heuristics = {list1[0]:1,list1[1]:2}


In [28]:
print("Start states:",start_states,"\nGoal:",goal)
print("heuristic functions:",heuristics)

Start states: [[4, 5, 1, 2, 3, 8, 0, 6, 7], [0, 1, 3, 5, 2, 6, 4, 7, 8], [1, 2, 3, 5, 6, 0, 7, 8, 4]] 
Goal: [[1, 2, 3, 4, 5, 6, 7, 8, 0], [1, 2, 3, 5, 8, 6, 0, 7, 4]]
heuristic functions: {'Misplaced Tiles': 1, 'Manhattan Distance': 2}


In [4]:
# A state is a simple list of 9 numbers, a permutation of 0-9. 0 represents the blank.

# Convert a 0-8 index into a row number
def row(pos):
        return (int(pos/3))

    # Convert a 0-8 index into a column number
def col(pos):
        return int(pos%3)

# Inverse conversion
def index(row, col):
        return row*3+col

    
class Puzzle:

    goal_state = None
    start_state = None
    hn = None                              # heuristic preference
        
    def __init__(self,state,parent,action):     # initiater of class object
        
        self.state = state
        self.parent = parent
        self.action = action

    # Returns a list of possible moves from a given state so that we can generate a random move.
    #The four possible moves are "U","D","L","R" for the four directions,
    #but the function should only return the subset of these that are possible in the current configuration.

    def all_moves(self):
        space = self.state.index(0) # where the 0 is
        r = row(space)
        c = col(space)
        moves = ["U", "D", "L", "R"]
        if r == 0:
            moves.remove("U")
        if r == 2:
            moves.remove("D")
        if c == 0:
            moves.remove("L")
        if c == 2:
            moves.remove("R")
        return moves

    # Execute a move from the given state. 
    def do_move(self,move):
        new_state = self.state.copy()
        space = new_state.index(0)
        r = row(space)
        c = col(space)
        if move == "U":
            move_index = index(r-1, c)
        if move == "D":
            move_index = index(r+1, c)
        if move == "R":
            move_index = index(r, c+1)
        if move == "L":
            move_index = index(r, c-1)
        # the next instruction is a swap!
        new_state[space],new_state[move_index] = new_state[move_index],new_state[space]
        return Puzzle(new_state,self,move)

    # calculate energy of a state based on given heuristic function preference
    def energy(self):
        
        sum1 = 0
        sum2 = 0
        
        for i in range(9):
            
            if self.state[i] == 0:
                continue
            else:
                sum1 += abs(self.state[i] - self.goal_state[i])
            
            if(self.state[i] != self.goal_state[i]): sum2 += 1
                
        
        if self.hn == 1:
            return sum1
        elif self.hn == 2:
            return sum2
        else:
            return (sum1*sum2)

        
    def solved(self):
        if(self.state == self.goal_state):
            return True
        else:
            return False
    
    def get_path(self):                    #find optimal path if goal_state reached
        
        solution = []
        solution.append(self.action)            #storing the action taken to come to this current state
        path = self
        while path.parent != None:              #backtracking to the start_state
            path = path.parent         
            solution.append(path.action)
        solution = solution[:-1]
        solution.reverse()
        return solution
    


In [5]:
def print_puzzle(state):
    for r in range(3):
        for c in range(3):
            print(state[index(r,c)], end = ' ')
        print()

In [11]:
# The simulated annealing algorithm. We only allow it to run for a
# given number of maximum moves. The parameter p is a control
# parameter for the rate between the energy and the temperature.
def sim_annealing(initial_state,goal_state, temp, hn):

    Puzzle.start_state = initial_state
    Puzzle.goal_state = goal_state
    Puzzle.hn = hn
    temperature = temp
    state = Puzzle(initial_state,None,None)   
#     print(state.state)
    
    oldE = state.energy()                     # getting initial state energy
    n_explored = 0
    flag =False                               # to remember whether converged to goal or not

    while temperature > 0:
        moves = state.all_moves()             # list of all moves possible
        accepted = False
        
        while not accepted:

            m = choice(moves)              # choosing a random move from given list of moves
            new_state = state.do_move(m)   # create a new state with choosen move
            n_explored+=1                  # increment the no. of explored state
            newE = new_state.energy()      # compute new state energy
            
            deltaE = newE - oldE           # compute difference in energy
            if deltaE <= 0:
                accepted = True
                state = new_state          # update accepted new state as current state
            else:
                boltz = exp(-float(deltaE)/temperature)# probability of accepting the bad state having larger energy value
                r = random()                           # generate a float random number between 0 and 1
                if r <= boltz:                         # this states accepting the state with some defined probabilty if true
                    accepted = True
                    state = new_state
        
        oldE = newE                         # update the energy if a state is accepted
        temperature = cool(temperature)     # lower the temperature
        
        if state.solved():                  # if converged return and mark flag as true
            flag = True
            break;
    
    opt_path = state.get_path()           # getting (sub)optimal path achieved
    return(opt_path,n_explored,flag)

In [30]:
#defining new heuristic function
h_1 = 'Misplaced Tiles'
h_2 = 'Manhattan Distance'
h_3 = "h3 = h1 * h2"
heuristics[h_3] = 3


In [31]:
# Setting initial temperature and cooling fucntion
temp = 100

cool_f = 'T = T-1'
def cool(T):
    return (T-1)

In [33]:
puzzle = start_states[2]
goal_puzzle = goal[1]

print("Start State")
print_puzzle(puzzle)
print("\nGoal State")
print_puzzle(goal_puzzle)

for hi in heuristics:
    print("\nHeuristics chosen:",hi,"\nTemperature chosen:",temp,"\nCooling function:",cool_f,"\n")
    
    t0 = time()
    result = sim_annealing(puzzle, goal_puzzle, temp, heuristics[hi])    # h_1: Manhattan , h_2: Misplaced, h_3: h3 = h1*h2
    t1 = time() - t0

    opt_path = result[0]
    explored = result[1]
    flag = result[2]

    if flag:
        print("Puzzle solved successfully\n")
        print('Optimal path:',opt_path)

    else:
        print("Puzzle cant be solved\n")

    print('Total no. of states explored: ',explored)
    print('\nTime taken: ', t1)
    print("\n\n")

Start State
1 2 3 
5 6 0 
7 8 4 

Goal State
1 2 3 
5 8 6 
0 7 4 

Heuristics chosen: Misplaced Tiles 
Temperature chosen: 100 
Cooling function: T = T-1 

Puzzle solved successfully

Optimal path: ['L', 'D', 'R', 'L', 'L']
Total no. of states explored:  5

Time taken:  0.0009968280792236328




Heuristics chosen: Manhattan Distance 
Temperature chosen: 100 
Cooling function: T = T-1 

Puzzle cant be solved

Total no. of states explored:  102

Time taken:  0.003991365432739258




Heuristics chosen: h3 = h1 * h2 
Temperature chosen: 100 
Cooling function: T = T-1 

Puzzle solved successfully

Optimal path: ['L', 'L', 'D', 'U', 'R', 'D', 'L']
Total no. of states explored:  8

Time taken:  0.0



